# **パフォーマンス評価**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## **1. ファイルからデータを読み込み**

In [5]:
import pandas as pd
import numpy as np

# Googleドライブのパスを指定して読み込み
file_path = '/content/drive/My Drive/csv/input/stock_price.csv'

df_stock = pd.read_csv(file_path)

# Date列を日付型に変換
df_stock['Date'] = pd.to_datetime(df_stock['Date'])

# 証券Codeごとにグループ化して収益率を計算
df_stock['Simple_Return'] = df_stock.groupby('Code')['Price'].pct_change()

# TOPIXリターンの読み込み
file_path = '/content/drive/My Drive/csv/input/market_return.csv'

df_market = pd.read_csv(file_path)

# Date列を日付型に変換
df_market['Date'] = pd.to_datetime(df_market['Date'])

## **2. NAを除去**

In [ ]:
# 日付をキーにしてレフトジョイン
df = pd.merge(df_stock, df_market, on='Date', how='left').dropna()

# 結果の表示
print(df.head())

## **3. ベータ、ジェンセンのアルファを計算**

In [ ]:
codeList = df['Code'].unique()
risk_free_rate = 0

# 各証券コードに対してベータ、ジェンセンのアルファを計算
alphas = {}
betas = {}
for code_ in codeList:

  df_code = df[df['Code'] == code_]
  stock_returns = df_code['Simple_Return'] - risk_free_rate
  market_returns = df_code['TOPIX'] - risk_free_rate

  beta = stock_returns.cov(market_returns)/market_returns.var()
  alpha = stock_returns.mean() - beta *(market_returns.mean() - risk_free_rate)

  alphas[code_] = alpha
  betas[code_] = beta

# 結果を表示
for code in alphas.keys():
    alpha = alphas[code]
    beta = betas[code]
    print(f"Code: {code}, Jensen's Alpha: {alpha:.4%},    Beta: {beta:.4f}")


## **4. シャープレシオを計算**

In [ ]:
codeList = df['Code'].unique()
risk_free_rate = 0

# 各証券コードに対してベータ、ジェンセンのアルファを計算
sharpe_ratios = {}
betas = {}
for code_ in codeList:

  df_code = df[df['Code'] == code_]
  stock_returns = df_code['Simple_Return']

  sharpe_ratio = (stock_returns.mean() - risk_free_rate) / stock_returns.std()

  sharpe_ratios[code_] = sharpe_ratio

# 結果を表示
for code in sharpe_ratios.keys():
    sharpe_ratio = sharpe_ratios[code]

    print(f"Code: {code}, Sharpe Ratio: {sharpe_ratio:.2f}")